In [ ]:
#| default_exp pds.apps

# Apps
> User interface apps/functions to make using the PDS tools easy.

In [ ]:
#| hide
from nbdev.showdoc import show_doc  # noqa

In [ ]:
#| export
import pandas as pd
from planetarypy.config import config
from planetarypy.pds.indexes import Index

In [ ]:
#| export
def find_indexes(
        instrument: str,  # Dotted mission.instrument key, e.g. cassini.iss
) -> list:  # List of configured index names
    "Find existing indexes for an instrument."
    return config.list_indexes(instrument)

In [ ]:
find_indexes("cassini.iss")

In [ ]:
#| export
def get_index(
        instr: str,  # Dotted instrument index, e.g. cassini.iss
        index_name: str = '',  # Index name, for exmample 'moon_summary. Optional'
        refresh: bool = False,  # switch to force a refresh of an index
) -> pd.DataFrame:  # The PDS index convert to pandas DataFrame
    """Example: get_index("cassini.iss", "index")"""
    if not index_name:
        index = Index(instr)
    else:
        index = Index(instr + ".indexes." + index_name)
    if not index.local_table_path.exists() or refresh:
        index.download()
    if index.update_available:
        print("An updated index is available.")
        print("Call `get_index` with `refresh=True` to get the updated version.")
    return index.parquet

In [ ]:
instr = "cassini.iss"
index_name = "moon_summary"

In [ ]:
index = get_index(instr, index_name)

In [ ]:
index

Indexes can be given as 2 parameters ("cassini.iss", "ring_summary") or one ("cassini.iss.ring_summary"):

In [ ]:
index = get_index("cassini.iss", "ring_summary")
index.head()

In [ ]:
index = get_index("cassini.iss.ring_summary")
index.head()

In [ ]:
#| export
def find_instruments(
        mission: str,  # Mission string, e.g. "cassini"
) -> list:  # List of configured instrument names
    "Find existing instruments for a mission."
    return config.list_instruments(mission)

In [ ]:
find_instruments("cassini")

In [ ]:
find_instruments("mro")

In [ ]:
df = get_index("cassini.uvis", "index")

In [ ]:
df.dtypes